In [ ]:
from fast_map import fast_map
import numpy as np
import scipy.sparse as sp
from sklearn.linear_model import ElasticNet
from tqdm import tqdm

from src.recommender_model import RecommenderModel
from src.utils import train_model

# SLIM Elastic Net


In [ ]:
class SLIMElasticNet(RecommenderModel):
	def __init__(self):
	   super(SLIMElasticNet, self).__init__()
	   self.alpha: float = 0
	   self.l1_ratio: float = 0
	   self.top_k: int = 0
	   self.max_iter: int = 0
	   self.similarity_matrix: sp.csr_matrix | None = None

	@staticmethod
	def process_item(item_idx: int, urm_csc: sp.csc_matrix, top_k: int, alpha: float, l1_ratio: float, max_iter: int):
		elastic_net = ElasticNet(
			alpha=alpha,
			l1_ratio=l1_ratio,
			fit_intercept=False,
			positive=True,
			copy_X=False,
			selection='random',
			max_iter=max_iter,
			tol=1e-4
		)

		y = urm_csc[:, item_idx].toarray()
		x = urm_csc.copy()
		x.data[x.indptr[item_idx]:x.indptr[item_idx + 1]] = 0.

		elastic_net.fit(urm_csc, y)

		coeffs_idxs = elastic_net.sparse_coef_.indices
		coeffs_vals = elastic_net.sparse_coef_.data

		if coeffs_idxs.shape[0] > top_k:
			relevant_items = np.argpartition(-np.abs(coeffs_vals), top_k)[:top_k]
			coeffs_idxs = coeffs_idxs[relevant_items]
			coeffs_vals = coeffs_vals[relevant_items]

		return item_idx, coeffs_idxs, coeffs_vals

	def fit(
		self,
		urm: sp.csr_matrix,
		icm: sp.csr_matrix = None,
		urm_val: sp.csr_matrix = None,
		top_k: int = 300,
		l1_reg: float = 1e-7,
		l2_reg: float = 1e-5,
		max_iter: int = 100
	) -> None:
		self.urm = urm
		urm_csc = self.urm.tocsr()
		num_items = self.urm.shape[1]

		self.top_k = min(top_k, num_items - 1)
		self.alpha = l1_reg + l2_reg
		self.l1_ratio = l1_reg / self.alpha
		self.max_iter = max_iter

		s_rows = []
		s_cols = []
		s_vals = []

		for item_idx, coeffs_idxs, coeffs_vals in tqdm(fast_map(self.process_item, range(num_items), [urm_csc] * num_items, [self.top_k] * num_items, [self.alpha] * num_items, [self.l1_ratio] * num_items, [self.max_iter] * num_items), desc="Items", total=num_items):
			s_rows.extend([item_idx] * len(coeffs_idxs))
			s_cols.extend(coeffs_idxs)
			s_vals.extend(coeffs_vals)

		self.similarity_matrix = sp.csr_matrix(
			(s_vals, (s_rows, s_cols)),
			shape=(num_items, num_items),
			dtype=np.float32
		)

		self.urm_pred = self.urm @ self.similarity_matrix

In [ ]:
mfmse_train = train_model(SLIMElasticNet())

Submission result: `0.00278`